In [ ]:
from os import environ
input_folder = environ.get('CROSSCOMPUTE_INPUT_FOLDER', 'tests/standard/input')
output_folder = environ.get('CROSSCOMPUTE_OUTPUT_FOLDER', 'tests/standard/output')
GOOGLE_KEY = environ['GOOGLE_KEY']

In [ ]:
import json
from os.path import join
variables = json.load(open(join(input_folder, 'variables.dictionary')))
description = variables['description']
location = variables['location']

In [ ]:
import requests
from time import sleep

def fetch_json(url):
    while True:
        response = requests.get(url)
        response_json = response.json()
        response_status = response_json['status']
        if response_status == 'OK':
            break
        elif response_status == 'REQUEST_DENIED':
            raise requests.exceptions.ConnectionError(
                'please check your GOOGLE_KEY')
        print(response_status)
        sleep(2)
    return response_json

endpoint_url = 'https://maps.googleapis.com/maps/api/place/textsearch/json'
text = f'{description} in {location}'
url = f'{endpoint_url}?query={text}&key={GOOGLE_KEY}'
response_json = fetch_json(url)

In [ ]:
results = []
results.extend(response_json['results'])
len(results)

In [ ]:
while 'next_page_token' in response_json:
    page_token = response_json['next_page_token']
    url = f'{endpoint_url}?pagetoken={page_token}&key={GOOGLE_KEY}'
    sleep(2)
    response_json = fetch_json(url)
    results.extend(response_json['results'])
    print(len(results))

In [ ]:
json.dump(results, open(join(output_folder, 'raw.json'), 'wt'))

In [ ]:
dictionaries = []
for r in results:
    l = r['geometry']['location']
    dictionaries.append({
        'name': r['name'],
        'address': r['formatted_address'],
        'status': r['business_status'],
        'rating': r['rating'],
        'rating_count': r['user_ratings_total'],
        'longitude': l['lng'],
        'latitude': l['lat'],
    })

In [ ]:
import csv
columns = [
    'name', 'address', 'status', 'rating', 'rating_count',
    'longitude', 'latitude']
rows = [[_[k] for k in columns] for _ in dictionaries]
with open(join(output_folder, 'places.csv'), 'wt') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(columns)
    csv_writer.writerows(rows)

In [ ]:
features = []
for d in dictionaries:
    features.append({
        'type': 'Feature',
        'properties': {
            'name': d['name'],
            'address': d['address'],
            'status': d['status'],
            'rating': d['rating'],
            'rating_count': d['rating_count'],
        },
        'geometry': {
            'type': 'Point',
            'coordinates': [d['longitude'], d['latitude']],
        },
    })
json.dump({
    'type': 'FeatureCollection',
    'features': features,
}, open(join(output_folder, 'geometries.geojson'), 'wt'))

In [ ]:
xys = []
for d in dictionaries:
    xys.append((d['longitude'], d['latitude']))
# json.dump(xys, open(join(output_folder, 'geometries.json'), 'wt'))

In [ ]:
def average(xs):
    return sum(xs) / len(xs)

json.dump({
    'longitude': average([_[0] for _ in xys]),
    'latitude': average([_[1] for _ in xys]),
    'zoom': 10,
}, open(join(output_folder, 'map.json'), 'wt'))